In [7]:
import csv
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import random
import string
import re
#tokenize and remove stopword
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english')) 

ImportError: cannot import name compat

In [ ]:
data1 = pd.read_csv('A:/PWR/project/AI/ATAI/dataset/dataset.csv')
book_title=data1.title;
book_summary = data1.summary;
book_genre=data1.genre;
#data1.head()

In [ ]:
newdf = data1[data1.columns[1:3]]
#newdf.head()

In [ ]:
data1[['title','summary','genre']].head()

In [ ]:
def removeNonAscii(s): 
    return "".join(i for i in s if ord(i)<128)

def strip_links(text):
    link_regex    = re.compile("([^-_a-zA-Z0-9!@#%&=,/'\";:~`\$\^\*\(\)\+\[\]\.\{\}\|\?\<\>\\]+|[^\s]+)")
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

def drop_digits(in_str):
    digit_list = "1234567890"
    for char in digit_list:
        in_str = in_str.replace(char, "")
    return in_str

def strip_all_entities(text):
    entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

In [ ]:
wacana=[]
for t in data1['title']:
    titel=strip_all_entities(drop_digits(strip_links(removeNonAscii(t.lower()))))
    wacana.append(titel)

#print(wacana)

In [ ]:
review=[]
for s in data1['summary']:
    titel=strip_all_entities(drop_digits(strip_links(removeNonAscii(s.lower()))))
    review.append(titel)

#print(review)

In [ ]:
book_clean = pd.DataFrame(dict(ide=data1.id, title=wacana, summary=review, genre=data1.genre)) 
book_clean[['ide','title','summary','genre']].head()

In [ ]:
book_vectorizer = TfidfVectorizer(max_df=0.9, min_df=0.1, stop_words='english')
book_matrix = book_vectorizer.fit_transform(book_clean.summary)
print 'There are '+ str(book_matrix.get_shape()[0]) + ' documents over a vocabulary feature space of ' + str(book_matrix.get_shape()[1]) + ' terms.'

tfidf = pd.DataFrame(book_matrix.toarray(), index=[book_clean.ide, book_clean.title, book_clean.genre], columns=book_vectorizer.get_feature_names())
#tfidf.to_csv('A:/PWR/project/AI/ATAI/dataset/bookdataset.csv')
tfidf.head()

In [ ]:
training_data=tfidf.iloc[:-150]
training_data.head()
#training_data.to_csv('A:/PWR/project/AI/ATAI/dataset/trainingdata2.csv')


In [ ]:
training_label=book_clean.genre.iloc[:-150]
#training_label.head()

In [ ]:
testing_data=tfidf.iloc[-150:]
#testing_data.to_csv('A:/PWR/project/AI/ATAI/dataset/testingdata2.csv')
testing_data.head()

In [ ]:
testing_label=book_clean.genre.iloc[-150:]
testing_title=book_clean.title.iloc[-150:]

In [ ]:
#print tfidf_matrix
from sklearn.metrics.pairwise import cosine_similarity
kosinus = cosine_similarity(training_data, testing_data)
#np.savetxt('A:/PWR/project/AI/ATAI/dataset/kos_test.txt', kosinus, newline="\n", delimiter=',', fmt='%1.6f')

In [ ]:
# mencari jarak/similarity dengan cosine
def distance(instance1, instance2):
    # buat jaga2 kalo instans nya adalah list atau tuple
    instance1 = np.array(instance1) 
    instance2 = np.array(instance2)
    kosin=cosine_similarity(instance1, instance2)
    return kosin

In [ ]:
#book_clean.title.values[i-1:i]

In [ ]:
distances=[]
count = 1
datashow = pd.DataFrame(columns=["index_test","index_train", "testing_title", "training_title", "distance", "genrets","genretr"])
while (count <= 3):
    time.sleep(0.1)
    i=1
    while (i<=100):
        dist=distance(training_data[i-1:i], testing_data[count-1:count])
        distances.append((count,i, list(book_clean.title.values[i-1:i]), dist, list(testing_label.values[count-1:count]), list(training_label.values[i-1:i])))
        #datashow = datashow.append({"index_test": count, "index_train": i, "testing_title":list(testing_title.values[count-1:count]), "training_title":list(book_clean.title.values[i-1:i]), "distance":dist,"genrets":list(testing_label.values[count-1:count]),"genretr":list(training_label.values[i-1:i]) }, ignore_index=True)
        i=i+1
        
    count = count + 1
    update_progress(i / number_of_elements)

    update_progress(1)
    
#datashow.to_csv('A:/PWR/project/AI/ATAI/dataset/distances3.csv')

In [ ]:
datashow

In [ ]:
import collections

c = collections.Counter()
f=sort.genretr
for line in f:
    c.update(line)

print 'Most common:'
for letter, count in c.most_common(3):
    print '%s: %7d' % (letter, count)

In [ ]:
indextest=1
while indextest<=5:
    c = collections.Counter()
    sort=datashow[datashow['index_test'] == indextest].sort_values(by=['distance'], ascending=False).iloc[0:100]
    f=sort.genretr
    for line in f:
        c.update(line)
    print 'class for data test:', indextest
    #s='Most common in title'
    
    for letter, count in c.most_common(1):
        print '%s: %7d' % (letter, count)
    indextest=indextest+1

In [ ]:
sort=datashow[datashow['index_test'] == 1].sort_values(by=['distance'], ascending=False).iloc[0:50]
sort